<!--
tags: ["RAG"]
description: |
    Explore the use of AutoGen's RagAgent for tasks like code generation from docstrings, answering complex questions with human feedback, and exploiting features like Update Context, custom prompts, and few-shot learning.
-->

# Using RagAgent for Retrieval-Augmented Code Generation and Question Answering

Introducing an agent capable of performing Retrieval-Augmented Generation (RAG) for the given message.

Upon receipt of a message, the agent employs RAG to generate a reply. It retrieves documents based on the message, then generates a reply using both the retrieved documents and the message itself. Additionally, it supports automatic context updates during the conversation, either autonomously or at the user`s request.

We also support enabling the RAG capability for any conversable agent with `Ragability`.

## Table of Contents
We'll demonstrate six examples of using RetrieveChat for code generation and question answering:

- [Example 1: Use RagAgent to generate code](#example-1)
- [Example 2: Use RagAgent to answer a multi-hop question](#example-2)
- [Example 3: Enable RAG capability for any conversable agent](#example-3)
- [Example 4: Complicate multi-round conversation](#example-4)

````{=mdx}
:::info Requirements
Some extra dependencies are needed for this notebook, which can be installed via pip:

```bash
pip install pyautogen[rag] flaml[automl]
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [1]:
import os
import sys
import logging
import autogen
from autogen.agentchat.contrib.rag import RagAgent, logger
from autogen.agentchat.contrib.rag.splitter import SUPPORTED_EXTENSIONS
from autogen.agentchat.contrib.capabilities.rag_capability import Ragability

logger.setLevel(logging.DEBUG)

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")

assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['gpt-35-turbo', 'gpt-35-turbo-0613']


````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/llm_configuration).
:::
````

## Construct agents for RAG

We start by initializing the `RagAgent` and `UserProxyAgent`. The `RagAgent` will be responsible for replying to `UserProxyAgent` with retrieval-augmented generation based on the given knowledge base contained in the `docs_path`.

In [2]:
print(f"Accepted file formats for `docs_path`: {SUPPORTED_EXTENSIONS}")

Accepted file formats for `docs_path`: ['.txt', '.json', '.csv', '.tsv', '.md', '.html', '.htm', '.rtf', '.rst', '.jsonl', '.log', '.xml', '.yaml', '.yml', '.pdf']


In [3]:
llm_config = {
    "timeout": 60,
    "config_list": config_list,
}


def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()


userproxy = autogen.UserProxyAgent(
    name="userproxy",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    code_execution_config=False,  # {"use_docker": False, "work_dir": ".tmp"},
    # default_auto_reply="Reply `TERMINATE` if the task is done.",
    description="The boss who ask questions and give tasks.",
)

# Check the docstring of RagAgent for more details on the configuration.
rag_config = {
    "docs_path": [
        "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
        "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md",
        os.path.join(os.path.abspath(""), "..", "website", "docs"),
    ],
    "overwrite": True,  # set to False to avoid overwriting the existing documents
}

rag = RagAgent(
    name="rag",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    llm_config=llm_config,
    rag_config=rag_config,
    code_execution_config=False,
    description="Assistant that can answer questions and generate code based on retrieved documents.",
)

/home/lijiang1/anaconda3/envs/autogen/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
2024-02-29 17:58:45,186 - utils.py:   16 - DEBUG - Processing file: ./tmp/download/Integrate%20-%20Spark.md, url: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md
2024-02-29 17:58:45,490 - utils.py:   16 - DEBUG - Split ./tmp/download/Integrate%20-%20Spark.md into 2 chunks.
2024-02-29 17:58:45,491 - utils.py:   16 - DEBUG - Processing file: ./tmp/download/Research.md, url: https:/

### Example 1
### Use RagAgent to generate code

[Back to top](#table-of-contents)

Use RagAgent to help generate sample code and automatically run the code and fix errors if there is any.

Problem: How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.

In [4]:
code_problem = "How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached."
rag.rag_filter_document = {"$contains": "spark"}  # filter documents that contain "spark"
_ = userproxy.initiate_chat(rag, message=code_problem)

userproxy (to rag):

How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.

--------------------------------------------------------------------------------


2024-02-29 17:58:49,519 - utils.py:   16 - DEBUG - is_code_execution_result: False
2024-02-29 17:58:49,520 - utils.py:   16 - DEBUG - Input message: How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.
2024-02-29 17:58:49,520 - utils.py:   16 - DEBUG - Received raw message: How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.
2024-02-29 17:58:49,521 - utils.py:   16 - DEBUG - Performing RAG for message: How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.
2024-02-29 17:58:49,541 - utils.py:   16 - DEBUG - Task predicted: `code`
2024-02-29 17:58:49,542 - utils.py:   16 - DEBUG - __call__ took 0.02 seconds.
2024-02-29 17:58:49,542 - utils.py:   16 - DEBUG - Refined message for d

rag (to userproxy):

Here's an example how to use FLAML with Spark to train and classify in 30 seconds and automatically cancel jobs if it exceeds the time limit:

```python
import flaml
from flaml.automl.spark.utils import to_pandas_on_spark
import pyspark.sql.functions as F
from pyspark.sql.functions import col

# Load data
df = spark.table("flights")
pdf = df.limit(100).toPandas()

# Convert to Pandas on Spark dataframe
psdf = to_pandas_on_spark(pdf)

# Specify label column name
label = "cancelled"

# Start AutoML experiment
automl = flaml.AutoML()

# Time budget and Spark settings
automl_settings = {
    "time_budget": 30,
    "metric": "accuracy",
    "n_concurrent_trials": 2,
    "use_spark": True,
    "force_cancel": True
}

# Train with AutoML
automl.fit(
    dataframe=psdf,
    label=label,
    **automl_settings,
)
```

This code specifies a 30-second time budget for the experiment. Spark is used as the parallel backend during parallel tuning, and if the search time exceeds th

2024-02-29 17:58:54,939 - utils.py:   16 - DEBUG - History message: {'content': 'How to use FLAML for a classification task and train the model in 30 seconds. Use spark to parallel the training. Force cancel jobs if time limit is reached.', 'role': 'user'}
2024-02-29 17:58:54,940 - utils.py:   16 - DEBUG - History message: {'content': 'Here\'s an example how to use FLAML with Spark to train and classify in 30 seconds and automatically cancel jobs if it exceeds the time limit:\n\n```python\nimport flaml\nfrom flaml.automl.spark.utils import to_pandas_on_spark\nimport pyspark.sql.functions as F\nfrom pyspark.sql.functions import col\n\n# Load data\ndf = spark.table("flights")\npdf = df.limit(100).toPandas()\n\n# Convert to Pandas on Spark dataframe\npsdf = to_pandas_on_spark(pdf)\n\n# Specify label column name\nlabel = "cancelled"\n\n# Start AutoML experiment\nautoml = flaml.AutoML()\n\n# Time budget and Spark settings\nautoml_settings = {\n    "time_budget": 30,\n    "metric": "accuracy

### Example 2
### Use RagAgent to answer a multi-hop question

[Back to top](#table-of-contents)

Use RagAgent to answer a multi-hop question.

Problem: The common authors of the paper FLAML and paper AutoGen.

In [5]:
# reset the assistant. Always reset the assistant before starting a new conversation.
rag.reset()

qa_problem = "The common authors of the paper FLAML and paper AutoGen."
rag.rag_filter_document = None  # clear filter
_ = userproxy.initiate_chat(rag, message=qa_problem)

userproxy (to rag):

The common authors of the paper FLAML and paper AutoGen.

--------------------------------------------------------------------------------


2024-02-29 17:58:54,966 - utils.py:   16 - DEBUG - is_code_execution_result: False
2024-02-29 17:58:54,966 - utils.py:   16 - DEBUG - Input message: The common authors of the paper FLAML and paper AutoGen.
2024-02-29 17:58:54,967 - utils.py:   16 - DEBUG - Received raw message: The common authors of the paper FLAML and paper AutoGen.
2024-02-29 17:58:54,968 - utils.py:   16 - DEBUG - Performing RAG for message: The common authors of the paper FLAML and paper AutoGen.
2024-02-29 17:58:54,990 - utils.py:   16 - DEBUG - Task predicted: qa
2024-02-29 17:58:54,991 - utils.py:   16 - DEBUG - __call__ took 0.02 seconds.
2024-02-29 17:58:54,992 - utils.py:   16 - DEBUG - Refined message for db query: ['Who authored both FLAML paper and AutoGen paper', 'Which authors contributed to both FLAML and AutoGen papers']
2024-02-29 17:58:55,168 - utils.py:   16 - DEBUG - retrieve_docs took 0.17 seconds.
2024-02-29 17:58:55,170 - utils.py:   16 - DEBUG - get_docs_by_ids took 0.00 seconds.
2024-02-29 17:

rag (to userproxy):

Answer is: Chi Wang, Qingyun Wu, Markus Weimer, and Erkang Zhu authored both FLAML paper and AutoGen paper.

Source: /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/agent_chat.md /home/lijiang1/code/autogen/notebook/../website/docs/FAQ.md /home/lijiang1/code/autogen/notebook/../website/docs/Ecosystem.md /home/lijiang1/code/autogen/notebook/../website/docs/Examples.md

--------------------------------------------------------------------------------
userproxy (to rag):



--------------------------------------------------------------------------------


2024-02-29 17:58:59,308 - utils.py:   16 - DEBUG - History message: {'content': 'The common authors of the paper FLAML and paper AutoGen.', 'role': 'user'}
2024-02-29 17:58:59,309 - utils.py:   16 - DEBUG - History message: {'content': 'Answer is: Chi Wang, Qingyun Wu, Markus Weimer, and Erkang Zhu authored both FLAML paper and AutoGen paper.\n\nSource: /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/agent_chat.md /home/lijiang1/code/autogen/notebook/../website/docs/FAQ.md /home/lijiang1/code/autogen/notebook/../website/docs/Ecosystem.md /home/lijiang1/code/autogen/notebook/../website/docs/Examples.md', 'role': 'assistant'}
2024-02-29 17:58:59,311 - utils.py:   16 - DEBUG - is_code_execution_result: False
2024-02-29 17:58:59,312 - utils.py:   16 - DEBUG - Input message: 
2024-02-29 17:58:59,313 - utils.py:   16 - DEBUG - Received raw message: The common authors of the paper FLAML and paper AutoGen.
2024-02-29 17:58:59,313 - utils.py:   16 - DEBUG - Is update context: Fal

### Example 3
### Enable RAG capability for any conversable agent

[Back to top](#table-of-contents)

Enable RAG capability for any conversable agent.

First, let's try to answer the same question in [Example 2](#example-2) with a default `AssistantAgent`. It couldn't answer it well because AutoGen paper is not in the LLM model's training data.

In [6]:
normal_assistant = autogen.AssistantAgent(name="normal assistant", llm_config=llm_config, max_consecutive_auto_reply=3)
message = "The common authors of the paper FLAML and paper AutoGen."

#### Normal AssistantAgent fails to answer the question

In [7]:
normal_assistant.reset()

_ = userproxy.initiate_chat(normal_assistant, message=message)

userproxy (to normal assistant):

The common authors of the paper FLAML and paper AutoGen.

--------------------------------------------------------------------------------
The common authors of the paper FLAML and paper AutoGen.

--------------------------------------------------------------------------------
normal assistant (to userproxy):

To find the common authors of the paper FLAML and paper AutoGen, we need to look for the authors who are listed in both the papers. We can do this by searching for the papers on Google Scholar and looking at the list of author names.

Here's the Python code to achieve this:
```python
# pip install scholarly
import scholarly

paper1 = "FLAML: A Python Library for Adapting and Learning Automatically from Multiple Data Streams"
paper2 = "AutoGen: Automatic Code Generation for Accelerating Deep Learning Applications"

# search for paper1 and get the first search result
search_results = scholarly.search_pubs(paper1)
pub1 = next(search_results)

# get 

Second, let's enable `Ragability` for the same assistant agent. As you can see, with the `Ragability` the agent can now answer the question correctly.

#### A normal AssistantAgent with Ragability successfully answers the question

In [8]:
rag_config["overwrite"] = False
ragability = Ragability(llm_config=llm_config, rag_config=rag_config, verbose=2)
ragability.add_to_agent(normal_assistant)

normal_assistant.reset()

_ = userproxy.initiate_chat(normal_assistant, message=message)

2024-02-29 17:59:01,919 - utils.py:   19 - INFO - Use the existing collection `autogen-rag`.
2024-02-29 17:59:01,921 - utils.py:   16 - DEBUG - insert_docs took 0.00 seconds.


userproxy (to normal assistant):

The common authors of the paper FLAML and paper AutoGen.

--------------------------------------------------------------------------------
normal assistant (to rag_agent):

The common authors of the paper FLAML and paper AutoGen.

--------------------------------------------------------------------------------


2024-02-29 17:59:01,937 - utils.py:   16 - DEBUG - is_code_execution_result: False
2024-02-29 17:59:01,938 - utils.py:   16 - DEBUG - Input message: The common authors of the paper FLAML and paper AutoGen.
2024-02-29 17:59:01,939 - utils.py:   16 - DEBUG - Received raw message: The common authors of the paper FLAML and paper AutoGen.
2024-02-29 17:59:01,940 - utils.py:   16 - DEBUG - Performing RAG for message: The common authors of the paper FLAML and paper AutoGen.
2024-02-29 17:59:01,957 - utils.py:   16 - DEBUG - Task predicted: qa
2024-02-29 17:59:01,957 - utils.py:   16 - DEBUG - __call__ took 0.02 seconds.
2024-02-29 17:59:01,958 - utils.py:   16 - DEBUG - Refined message for db query: ['Who authored both FLAML paper and AutoGen paper', 'Which authors contributed to both FLAML and AutoGen papers']
2024-02-29 17:59:02,065 - utils.py:   16 - DEBUG - retrieve_docs took 0.11 seconds.
2024-02-29 17:59:02,068 - utils.py:   16 - DEBUG - get_docs_by_ids took 0.00 seconds.
2024-02-29 17:

rag_agent (to normal assistant):

Answer is: Chi Wang, Qingyun Wu, Markus Weimer, and Erkang Zhu authored both FLAML paper and AutoGen paper.

Source: /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/agent_chat.md /home/lijiang1/code/autogen/notebook/../website/docs/FAQ.md /home/lijiang1/code/autogen/notebook/../website/docs/Ecosystem.md /home/lijiang1/code/autogen/notebook/../website/docs/Examples.md

--------------------------------------------------------------------------------
Answer is: Chi Wang, Qingyun Wu, Markus Weimer, and Erkang Zhu authored both FLAML paper and AutoGen paper.

Source: /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/agent_chat.md /home/lijiang1/code/autogen/notebook/../website/docs/FAQ.md /home/lijiang1/code/autogen/notebook/../website/docs/Ecosystem.md /home/lijiang1/code/autogen/notebook/../website/docs/Examples.md

--------------------------------------------------------------------------------
normal assistant (to userproxy):

### Example 4
### Complicate multi-round conversation

[Back to top](#table-of-contents)

Let's try a multi-round conversation. Uncomment below code to try it.

The questions from `human input` in the example are:

- what's autogen
- what's flaml
- what's the common authors of the two?
- List three papers they have co-authored?

In [14]:
# # reset the assistant. Always reset the assistant before starting a new conversation.
# rag.reset()
# rag.rag_filter_document = None  # clear filter
# rag.prompt_rag = autogen.agentchat.contrib.rag.prompts.PROMPT_QA
# userproxy.human_input_mode = "ALWAYS"
# _ = userproxy.initiate_chat(rag)

userproxy (to rag):

what's autogen

--------------------------------------------------------------------------------


2024-02-29 18:11:33,759 - utils.py:   16 - DEBUG - is_code_execution_result: False
2024-02-29 18:11:33,761 - utils.py:   16 - DEBUG - Input message: what's autogen
2024-02-29 18:11:33,761 - utils.py:   16 - DEBUG - Received raw message: what's autogen
2024-02-29 18:11:33,762 - utils.py:   16 - DEBUG - Performing RAG for message: what's autogen
2024-02-29 18:11:34,943 - utils.py:   16 - DEBUG - Task predicted: unknown
2024-02-29 18:11:34,944 - utils.py:   16 - DEBUG - __call__ took 1.18 seconds.
2024-02-29 18:11:34,945 - utils.py:   16 - DEBUG - Refined message for db query: ['What is autogen', 'What is the meaning of autogen']
2024-02-29 18:11:35,067 - utils.py:   16 - DEBUG - retrieve_docs took 0.12 seconds.
2024-02-29 18:11:35,070 - utils.py:   16 - DEBUG - get_docs_by_ids took 0.00 seconds.
2024-02-29 18:11:35,078 - utils.py:   16 - DEBUG - rerank took 0.01 seconds.
2024-02-29 18:11:35,084 - utils.py:   16 - DEBUG - get_docs_by_ids took 0.00 seconds.
2024-02-29 18:11:35,088 - utils.

rag (to userproxy):

Autogen is a framework that enables next-gen LLM(Large Language Model) applications via a Multi-Agent Conversation Framework. The details of the research and the framework are provided in these publications: [AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation Framework](https://arxiv.org/abs/2308.08155). Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Shaokun Zhang, Erkang Zhu, Beibin Li, Li Jiang, Xiaoyun Zhang and Chi Wang. ArXiv 2023.

Source: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/agent_chat.md /home/lijiang1/code/autogen/notebook/../website/docs/Ecosystem.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/enhanced_inference.md

--------------------------------------------------------------------------------
userproxy (to rag):

what's flaml

--------------------------------------------------------------------------------

2024-02-29 18:11:50,417 - utils.py:   16 - DEBUG - History message: {'content': "what's autogen", 'role': 'user'}
2024-02-29 18:11:50,417 - utils.py:   16 - DEBUG - History message: {'content': 'Autogen is a framework that enables next-gen LLM(Large Language Model) applications via a Multi-Agent Conversation Framework. The details of the research and the framework are provided in these publications: [AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation Framework](https://arxiv.org/abs/2308.08155). Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Shaokun Zhang, Erkang Zhu, Beibin Li, Li Jiang, Xiaoyun Zhang and Chi Wang. ArXiv 2023.\n\nSource: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/agent_chat.md /home/lijiang1/code/autogen/notebook/../website/docs/Ecosystem.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/enhanced_inference.md', 'role': 'assistant'

rag (to userproxy):

FLAML is an AutoML (Automatic Machine Learning) library developed by Microsoft that provides a fast and lightweight way of performing hyperparameter optimization for machine learning models. Autogen utilizes FLAML to handle hyperparameter tuning for its models. Specifically, Autogen uses FLAML to fine-tune its underlying LLM models to generate more accurate responses while minimizing cost and inference time.

Source: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/enhanced_inference.md

--------------------------------------------------------------------------------
userproxy (to rag):

what's the common authors of the two?

--------------------------------------------------------------------------------


2024-02-29 18:12:13,110 - utils.py:   16 - DEBUG - History message: {'content': "what's autogen", 'role': 'user'}
2024-02-29 18:12:13,111 - utils.py:   16 - DEBUG - History message: {'content': 'Autogen is a framework that enables next-gen LLM(Large Language Model) applications via a Multi-Agent Conversation Framework. The details of the research and the framework are provided in these publications: [AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation Framework](https://arxiv.org/abs/2308.08155). Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Shaokun Zhang, Erkang Zhu, Beibin Li, Li Jiang, Xiaoyun Zhang and Chi Wang. ArXiv 2023.\n\nSource: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/agent_chat.md /home/lijiang1/code/autogen/notebook/../website/docs/Ecosystem.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/enhanced_inference.md', 'role': 'assistant'

rag (to userproxy):

The common authors of Autogen and Flaml are Chi Wang and Qingyun Wu. They work on the development of both Autogen and Flaml, as can be seen from their research publications.

Source: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/enhanced_inference.md

Source: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/enhanced_inference.md

--------------------------------------------------------------------------------
userproxy (to rag):

List three papers they have co-authored?

--------------------------------------------------------------------------------


2024-02-29 18:13:23,675 - utils.py:   16 - DEBUG - History message: {'content': "what's autogen", 'role': 'user'}
2024-02-29 18:13:23,676 - utils.py:   16 - DEBUG - History message: {'content': 'Autogen is a framework that enables next-gen LLM(Large Language Model) applications via a Multi-Agent Conversation Framework. The details of the research and the framework are provided in these publications: [AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation Framework](https://arxiv.org/abs/2308.08155). Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Shaokun Zhang, Erkang Zhu, Beibin Li, Li Jiang, Xiaoyun Zhang and Chi Wang. ArXiv 2023.\n\nSource: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/agent_chat.md /home/lijiang1/code/autogen/notebook/../website/docs/Ecosystem.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/enhanced_inference.md', 'role': 'assistant'

rag (to userproxy):

The common authors of Autogen and Flaml have collaborated on several publications. Three of these publications are:

1. FLAML: A Fast and Lightweight AutoML Library. Chi Wang, Qingyun Wu, Markus Weimer, Erkang Zhu. MLSys 2021.
2. Economical Hyperparameter Optimization With Blended Search Strategy. Chi Wang, Qingyun Wu, Silu Huang, Amin Saied. ICLR 2021.
3. An Empirical Study on Hyperparameter Optimization for Fine-Tuning Pre-trained Language Models. Susan Xueqing Liu, Chi Wang. ACL 2021.

Source: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/agent_chat.md /home/lijiang1/code/autogen/notebook/../website/docs/Ecosystem.md /home/lijiang1/code/autogen/notebook/../website/docs/Use-Cases/enhanced_inference.md

Source: https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Int